In [1]:
import sys
import os
sys.path.append("/".join(os.getcwd().split("/")[:-1]))

In [3]:
from src.utils import *

/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/az01640/P

In [57]:
positives, backgrounds = load_raw_audio()

In [58]:
background = backgrounds[0]

In [166]:
def create_one_example(i_label, cut_background, positive, targeted_duration, n_classes):
    y = np.zeros(n_classes)
    y[i_label] = 1
    cut_background = match_target_amplitude(cut_background, -45)
    x = cut_background.overlay(positive, 
                               position=np.random.randint(low=0, high=targeted_duration - len(positive)))
    
    x = np.array(x.get_array_of_samples())
    x = np.swapaxes(get_spectrogram(x), 0, 1)
    
    return x.reshape(-1), y.reshape(-1)

In [172]:
def create_examples(positives, background, n_examples, targeted_duration, labels):
    labels_to_pick = random.choices(labels, k = n_examples)
    random_picks = np.random.randint(low=0, high=len(background) - targeted_duration, size=n_examples)
    background_picks = [background[r:r+targeted_duration-1] for r in random_picks]
    result = []
    
    for (i_label, label), background_pick in zip(labels_to_pick, background_picks):
        positive = random.choice(positives[label])
        result.append(create_one_example(i_label, background_pick, positive, targeted_duration, len(labels)))
    
    return result
    

In [173]:
def create_one_tf_record(data_dir, positives, background, targeted_duration):
    hashcode = uuid.uuid4()

    result_tf_file = "{}/{}.tfrecord".format(data_dir, hashcode)

    writer = tf.io.TFRecordWriter(result_tf_file)
    
    labels = list(enumerate(sorted(positives.keys())))
    
    n_samples = 100
    
    examples = create_examples(positives, background, n_samples, targeted_duration, labels)
    
    [writer.write(serialize_example(x, y)) for x, y in examples]
    
    writer.close()

In [177]:
from src.settings import *
import uuid

In [178]:
create_one_tf_record(DEV_CLASSIF_PROCESSED_DATA_DIR, positives, background, 2000)